In [1]:
import functions
import matplotlib.pyplot as plt
import pandas as pd
import torch

In [2]:
DATASET_PATH = "/scratch/kirschstein/LamaH-CE"
CHECKPOINT_DIR = "./runs/topology"

In [3]:
adj_df = pd.read_csv(f"{DATASET_PATH}/processed/adjacency.csv")
weight_cols = adj_df[["dist_hdn", "elev_diff", "strm_slope"]].values
stream_length = torch.tensor(weight_cols[:, 0], dtype=torch.float)
elevation_difference = torch.tensor(weight_cols[:, 1], dtype=torch.float)
average_slope = torch.tensor(weight_cols[:, 2], dtype=torch.float)

In [4]:
for architecture in ["ResGCN", "GCNII"]:
    for edge_orientation in ["downstream", "upstream", "bidirectional"]:
        print(architecture, edge_orientation)
        corrmats = []
        for fold in range(6):
            chkpt = torch.load(f"{CHECKPOINT_DIR}/{architecture}_{edge_orientation}_learned_{fold}.run")
            learned_weights = chkpt["history"]["best_model_params"]["edge_weights"].nan_to_num().cpu()
            corrmats.append(torch.corrcoef(torch.stack([learned_weights, stream_length, elevation_difference, average_slope]))) 
        print("correlation matrix mean:")
        print(torch.stack(corrmats).mean(dim=0))
        print("correlation matrix std:")
        print(torch.stack(corrmats).std(dim=0))
        print()

ResGCN downstream
correlation matrix mean:
tensor([[ 1.0000, -0.2213,  0.1000,  0.1683],
        [-0.2213,  1.0000,  0.3331, -0.1741],
        [ 0.1000,  0.3331,  1.0000,  0.5844],
        [ 0.1683, -0.1741,  0.5844,  1.0000]])
correlation matrix std:
tensor([[6.1559e-08, 9.8565e-02, 2.1019e-02, 3.8168e-02],
        [9.8565e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.1019e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [3.8168e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00]])

ResGCN upstream
correlation matrix mean:
tensor([[ 1.0000,  0.0422, -0.3080, -0.2930],
        [ 0.0422,  1.0000,  0.3331, -0.1741],
        [-0.3080,  0.3331,  1.0000,  0.5844],
        [-0.2930, -0.1741,  0.5844,  1.0000]])
correlation matrix std:
tensor([[0.0000, 0.0075, 0.0145, 0.0092],
        [0.0075, 0.0000, 0.0000, 0.0000],
        [0.0145, 0.0000, 0.0000, 0.0000],
        [0.0092, 0.0000, 0.0000, 0.0000]])

ResGCN bidirectional
correlation matrix mean:
tensor([[ 1.0000, -0.0023, -0.2353, -0.2400],
  